In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:15 http://ppa.launchpad

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-11-21 02:01:59--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.23MB/s    in 0.8s    

2021-11-21 02:02:00 (1.23 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://imdb-ratings-project-2021.s3.us-east-2.amazonaws.com/IMDb_Lang.csv"
spark.sparkContext.addFile(url)

In [6]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
IMDb_Lang = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), header=True, inferSchema=True)
IMDb_Lang.show()

+-------------+---------+--------+-----------+---------------+--------+-----+----------+------------------+--------------------+
|imdb_title_id|   genre1|duration|    country|language_movies|avg_vote|votes|    budget|reviews_from_users|reviews_from_critics|
+-------------+---------+--------+-----------+---------------+--------+-----+----------+------------------+--------------------+
|    tt0000009|  Romance|      45|        USA|           None|     5.9|  154|      null|                 1|                   2|
|    tt0000574|Biography|      70|  Australia|           None|     6.1|  589|    $2,250|                 7|                   7|
|    tt0001892|    Drama|      53|        USA|        English|     5.8|  188|      null|                 5|                   2|
|    tt0002101|Adventure|     100|      Italy|        Italian|     5.2|  446|   $45,000|                25|                   3|
|    tt0002130|Biography|      68|        USA|        English|     7.0| 2237|      null|         

In [7]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://movies.cuqh3xl6sha7.us-east-2.rds.amazonaws.com:5432/postgres"
config = {"user":"IMdb_movies", 
          "password": "11062011", 
          "driver":"org.postgresql.Driver"}

In [9]:
# Write customers_df to table in RDS
# 5 min 14 s
IMDb_Lang.write.jdbc(url=jdbc_url, table='IMDb_Lang', mode=mode, properties=config)